# **Tugas (LSA Topic Modelling)**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read Data

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Data didapatkan dari proses scrapping data youtube. Proses scrapping dapat dilihat pada halaman berikut (scrapping).

In [ ]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/sholehhuddin/data-csv/main/datacomment_cnn_fix.csv')
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nKoalisi Pendukun...,id skip navigation sign ini koalisi pendukung ...
1,Nasib Partai Nasdem di koalisi pemerintahan Jo...,nasib partai nasdem di koalisi pemerintahan jo...
2,Politik itu pilihan dan kepentingan dari parta...,politik itu pilihan dan kepentingan dari parta...
3,Kalaupun benar retak tentunya wajar. Ada orang...,kalaupun benar retak tentunya wajar ada orang ...
4,negara sdh amburadul kok msh mau d atur dgn ca...,negara sudah amburadul kok masih mau di atur d...
...,...,...
196,Karena pak jokowi selalu di celah anis basweda...,karena pak jokowi selalu di celah anis basweda...
197,Kasian nasdrun .kurang peka pada sinyal \n,kasihan nasdrun kurang peka pada sinyal
198,Koalisi Pendukung Jokowi Mulai Retak?\nCNN Ind...,koalisi pendukung jokowi mulai retak cn indone...
199,Nasib Partai Nasdem di koalisi pemerintahan Jo...,nasib partai nasdem di koalisi pemerintahan jo...


## Modelling

In [ ]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

## Bobot kata terhadap masing masing topik

In [ ]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('nasdem', 0.6246143745275621), ('jokowi', 0.32096023486217523)]
Topic 1:  [('rakyat', 0.5383726378908907), ('partai', 0.36061188484724443)]
Topic 2:  [('koalisi', 0.33101933164612135), ('presiden', 0.3208960926993904)]
Topic 3:  [('surya', 0.4683933502261465), ('paloh', 0.40094591971751325)]


## Bobot setiap topik terhadap dokumen

In [ ]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id skip navigation sign ini koalisi pendukung ...,0.302104,0.063721,0.040003,0.013245,0
1,nasib partai nasdem di koalisi pemerintahan jo...,0.429585,0.011216,0.591743,-0.356091,2
2,politik itu pilihan dan kepentingan dari parta...,0.276378,-0.061394,-0.153368,-0.089743,0
3,kalaupun benar retak tentunya wajar ada orang ...,0.160557,0.061311,0.073362,0.126429,0
4,negara sudah amburadul kok masih mau di atur d...,0.021363,0.019103,0.047198,0.021212,2
...,...,...,...,...,...,...
196,karena pak jokowi selalu di celah anis basweda...,0.131440,0.026169,0.077868,0.107238,0
197,kasihan nasdrun kurang peka pada sinyal,0.013509,-0.004676,-0.008389,-0.021149,0
198,koalisi pendukung jokowi mulai retak cn indone...,0.444314,0.013124,0.606413,-0.339728,2
199,nasib partai nasdem di koalisi pemerintahan jo...,0.429585,0.011216,0.591743,-0.356091,2


In [ ]:
df_lsa['Topik'].value_counts()

0    119
1     42
2     20
3     20
Name: Topik, dtype: int64